In [22]:
import numpy as np
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn import preprocessing, cross_validation
from sklearn.metrics import log_loss
from sklearn.metrics.classification import accuracy_score
from cmath import sqrt
import math


from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier 

In [5]:
data_crimes_train = pd.read_csv('train.csv', parse_dates=['Dates'])
data_crimes_test = pd.read_csv('test.csv', parse_dates=['Dates'])

In [6]:
data_crimes_train.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [7]:
data_crimes_test.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


In [8]:
data_crimes_train.describe()

,X,Y
count,878049.000000,878049.000000
mean,-122.422616,37.771020
std,0.030354,0.456893
min,-122.513642,37.707879
25%,-122.432952,37.752427
50%,-122.416420,37.775421
75%,-122.406959,37.784369
max,-120.500000,90.000000


In [9]:
list(set(data_crimes_train.Category.values))

['ARSON',
 'TREA',
 'STOLEN PROPERTY',
 'TRESPASS',
 'WARRANTS',
 'VEHICLE THEFT',
 'EXTORTION',
 'RUNAWAY',
 'BRIBERY',
 'PROSTITUTION',
 'DRUG/NARCOTIC',
 'ROBBERY',
 'SUSPICIOUS OCC',
 'SUICIDE',
 'FRAUD',
 'GAMBLING',
 'SECONDARY CODES',
 'SEX OFFENSES NON FORCIBLE',
 'FORGERY/COUNTERFEITING',
 'LIQUOR LAWS',
 'ASSAULT',
 'BAD CHECKS',
 'SEX OFFENSES FORCIBLE',
 'VANDALISM',
 'LARCENY/THEFT',
 'EMBEZZLEMENT',
 'BURGLARY',
 'DRUNKENNESS',
 'DRIVING UNDER THE INFLUENCE',
 'OTHER OFFENSES',
 'MISSING PERSON',
 'FAMILY OFFENSES',
 'PORNOGRAPHY/OBSCENE MAT',
 'NON-CRIMINAL',
 'RECOVERED VEHICLE',
 'LOITERING',
 'DISORDERLY CONDUCT',
 'KIDNAPPING',
 'WEAPON LAWS']

In [10]:
len(data_crimes_train.Address.unique())

23228

In [11]:
valuesPdDistrict = list(set(data_crimes_train.PdDistrict.values))
print(valuesPdDistrict)
valuesDayOfWeek = list(set(data_crimes_train.DayOfWeek.values))
print(valuesDayOfWeek)

['TENDERLOIN', 'PARK', 'CENTRAL', 'TARAVAL', 'RICHMOND', 'INGLESIDE', 'BAYVIEW', 'SOUTHERN', 'MISSION', 'NORTHERN']
['Thursday', 'Saturday', 'Tuesday', 'Friday', 'Monday', 'Wednesday', 'Sunday']


In [12]:
le_crime = preprocessing.LabelEncoder()
crime = le_crime.fit_transform(data_crimes_train.Category)

In [13]:
crime

array([37, 21, 21, ..., 16, 35, 12], dtype=int64)

In [14]:
days = pd.get_dummies(data_crimes_train.DayOfWeek)
district = pd.get_dummies(data_crimes_train.PdDistrict)
hour = data_crimes_train.Dates.dt.hour
hour = pd.get_dummies(hour)

In [15]:
train_data = pd.concat([hour, days,district], axis=1)
train_data['crime'] = crime

In [16]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN,crime
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,37
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,21
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,21
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,16
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,16


In [17]:
days = pd.get_dummies(data_crimes_test.DayOfWeek)
district = pd.get_dummies(data_crimes_test.PdDistrict)
hour = data_crimes_test.Dates.dt.hour
hour = pd.get_dummies(hour)  

test_data = pd.concat([hour, days, district], axis = 1)

In [18]:
features = valuesPdDistrict + valuesDayOfWeek + [x for x in range(0, 24)]
print(features)

['TENDERLOIN', 'PARK', 'CENTRAL', 'TARAVAL', 'RICHMOND', 'INGLESIDE', 'BAYVIEW', 'SOUTHERN', 'MISSION', 'NORTHERN', 'Thursday', 'Saturday', 'Tuesday', 'Friday', 'Monday', 'Wednesday', 'Sunday', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [19]:
training, validation = train_test_split(train_data, train_size=.70)

In [25]:
def run_model(model, training, validation):
    model.fit(training[features], training['crime'])
    predicted = np.array(model.predict_proba(validation[features]))
    print(("Log loss ")+str(log_loss(validation['crime'], predicted)))
    return model

In [26]:
modelBernoulliNB = run_model(BernoulliNB(), training, validation)

Log loss 2.58119170005


In [27]:
modelLogisticRegression = run_model(LogisticRegression(), training, validation)

Log loss 2.57898944425


In [28]:
modelMultinomialNB = run_model(MultinomialNB(), training, validation)

Log loss 2.57906841666


In [30]:
modelRandomForestClassifier = run_model(RandomForestClassifier(n_estimators=50, max_depth=5), training, validation)

Log loss 2.62363014359


In [31]:
#Se escoge el modelo realizado con LogisticRegression#

In [32]:
predicted = modelLogisticRegression.predict_proba(test_data[features])
result = pd.DataFrame(predicted, columns = le_crime.classes_)
result.to_csv('submit.csv',index=True, index_label='Id')